#### SPI10 2 NRS 

In [7]:
import torch
from torch.nn import functional as F
import pandas as pd


def nrs2class(x,pain_type=3):
    pain = ''
    
    if pain_type == 3:
        if x in [0,1,2,3]:
            pain = 0
        elif x in [4,5]:
            pain = 1
        else:
            pain = 2
    elif pain_type == 2:
        if x in [0,1,2,3]:
            pain = 0
        else:
            pain = 1
    return pain

class Spi10_2nrs(torch.utils.data.Dataset):
    def __init__(self,file_path='../data/spi10_2nrs.csv'):
        df = pd.read_csv(file_path)
        del df['id']
        df['10min'] = df['10min'].map(float)
        df['20min'] = df['20min'].map(float)
        df['30min'] = df['30min'].map(float)
        df['40min'] = df['40min'].map(float)
        df['50min'] = df['50min'].map(float)
        df['pacu_nrs'] = df.pacu_nrs.map(lambda x: nrs2class(x,2))
        self.df     = df

    def __getitem__(self,idx):
        x = torch.tensor(self.df.iloc[idx][:-1].to_numpy())
        y = torch.tensor([int(self.df.iloc[idx][-1])])
        y = F.one_hot(y,num_classes=2)
        return x,y
    
    def __len__(self):
        return len(self.df)


In [1]:
from dataset import Spi10_2nrs

In [3]:
import torch
trdt  = Spi10_2nrs()
trdl  = torch.utils.data.DataLoader(trdt, batch_size=36)

# test
trit = iter(trdl)

x,y = next(trit)

#### PPG 

In [9]:
import os
dir_path = '../data/vital2/'

flist = os.listdir(dir_path)

In [13]:
flist = [ x for x in x.endswith('.vital')

True

In [10]:
fpaths = [os.path.join(dir_path, x) for x in flist]

In [ ]:
import vitaldb
vf = vitaldb.VitalFile(fpaths[0])

In [ ]:
import torch
from torch.nn import functional as F
import pandas as pd
class ProteinDataset(torch.utils.data.Dataset):
    def __init__(self,path,transform='oneHot',only=None):
        self.df        = pd.read_csv(path)
        if type(transform) == str:
            self.transform = transform.lower()
        else:
            self.transform = transform
        
        if self.transform == 'ae':
            if only is None:
                only = 1
        
        if only is not None:
            self.df = self.df[self.df.label == only]
            
    def __getitem__(self,idx):
        item = self.df.iloc[idx]
        x = item['seq']
        y = item['label']
        
        if self.transform == 'onehot':
            x = self.seq2oneHot(x)
        elif self.transform is None:
            x = self.seq2int(x)
        elif self.transform == 'ae':
            x = self.seq2oneHot(x)
            x = x.to(torch.float)
            y = x 
            return x,y
        y = torch.tensor(y)
        # y = self.label2oneHot(y)
        return x,y
    
    def __len__(self):
        return len(self.df)
    


In [ ]:
vf

In [ ]:
# *.vital data
# check interval (1sec, or?)
# preprocess
# dataset
from torch import nn
class SimpleNet(nn.Module):
    def __init__(self,in_node=5,out_node=2):
        super(SimpleNet,self).__init__()
        
        self.backbone = torchvision.models.squeezenet1_0(pretrained=False)
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
            nn.Flatten(),
            nn.Linear(256,1)
          )
    def forward(self,x):
        x = self.backbone(x)
        return x

In [ ]:
import torch
from torch.nn import functional as F
import pandas as pd
class ProteinDataset(torch.utils.data.Dataset):
    def __init__(self,path,transform='oneHot',only=None):
        self.df        = pd.read_csv(path)
        if type(transform) == str:
            self.transform = transform.lower()
        else:
            self.transform = transform
        
        if self.transform == 'ae':
            if only is None:
                only = 1
        
        if only is not None:
            self.df = self.df[self.df.label == only]
            
    def __getitem__(self,idx):
        item = self.df.iloc[idx]
        x = item['seq']
        y = item['label']
        
        if self.transform == 'onehot':
            x = self.seq2oneHot(x)
        elif self.transform is None:
            x = self.seq2int(x)
        elif self.transform == 'ae':
            x = self.seq2oneHot(x)
            x = x.to(torch.float)
            y = x 
            return x,y
        y = torch.tensor(y)
        # y = self.label2oneHot(y)
        return x,y
    
    def __len__(self):
        return len(self.df)
    
